# Generate SSH key pairs to get connected to the Encord platform






In [ ]:
from IPython.display import display, Markdown

!ssh-keygen -t ed25519 -f ssh_key -N ''
with open('/content/ssh_key.pub', 'r') as f:
  public_key = f.read()

In [ ]:
display(Markdown(f'''
# Steps to register public SSH key to encord
1. Open [Encord](https://app.encord.com).
2. Go to **User => Settings => Public keys => + New key**.
3. Write   
   **Temporary Colab Key**   
   as title and paste  
   {public_key}    
   to the public key area.
4. Now, you can import your projects.

'''))


In [ ]:
!python -m pip install -q encord
from encord import EncordUserClient
from pathlib import Path

client = EncordUserClient.create_with_ssh_private_key(Path('/content/ssh_key').expanduser().read_text())
projects = client.get_projects()

display(Markdown(f'''
---
# Choose project from the below list:
---
'''))

for i, project in enumerate(projects):
  print(f'{i+1}. {project["project"]["title"]} ')
  
project_no = int(input('Project no:'))-1
project_name = projects[project_no-1]["project"]["title"]
project_hash = projects[project_no-1]["project"]["project_hash"]

# Open an [ngrok](https://dashboard.ngrok.com/signup) account, and paste your authtoken to the below cell

In [ ]:
!pip install -q pyngrok
import getpass
from pyngrok import ngrok, conf

# From ngrok dashboard, copy your authtoken and paste it here
ngrok.set_auth_token(<YOUR_AUTHTOKEN>)
for tunnel in ngrok.get_tunnels():
    ngrok.disconnect(tunnel.public_url)

# Open a tunnel to the HTTP server
connection_string = ngrok.connect(8501, headers={'ngrok-skip-browser-warning':'1'}).public_url

display(Markdown(f'''
---
---
---
   
# Click [this link]({connection_string}) and wait until this notebook finishes running, then refresh the page.
   
---
---
---
'''
))

## Install python 3.9, acivate it, and install Encord-Active

In [ ]:
!sudo apt-get -qq install python3.9 -q
!sudo apt -qq install python3.9-distutils -q
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 10

!sudo apt -qq install -q python3-pip
!python -m pip install -q --upgrade pip
!python -m pip install -q --upgrade setuptools wheel
!python -m pip install -q encord-active

## Start the app

In [ ]:
!encord-active config set ssh_key_path '/content/ssh_key'
!encord-active import project --project_hash=$project_hash
!encord-active visualise --target=$project_name}